# Package for WRC Results Scrape

In [159]:
url_base='http://www.wrc.com/service/sasCacheApi.php?route={stub}'

In [160]:
#Call a resource by ID
wrcapi='https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byId?id=%22{}%22' #requires resource ID

In [161]:
stubs = { 'itinerary': 'rallies/{rallyId}/itinerary',
          'startlists': 'rallies/{rallyId}/entries',
         'penalties': 'rallies/{rallyId}/penalties',
         'retirements': 'rallies/{rallyId}/retirements',
         'stagewinners':'rallies/{rallyId}/stagewinners',
         'overall':'stages/{stageId}/results?rallyId={rallyId}',
         'split_times':'stages/{stageId}/splittimes?rallyId={rallyId}',
         'stage_times_stage':'stages/{stageId}/stagetimes?rallyId={rallyId}',
         'stage_times_overall':'stages/{stageId}/results?rallyId={rallyId}',
         'championship':'championships/{championshipId}',
         'championship_results':'championships/{championshipId}/results' }

In [162]:
import requests
import re
import json
from bs4 import BeautifulSoup 

import sqlite3

import pandas as pd
from pandas.io.json import json_normalize

In [222]:
#SQL in wrcResults.sql
setup_q='''
CREATE TABLE "itinerary_event" (
  "eventId" INTEGER,
  "itineraryId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "priority" INTEGER
);
CREATE TABLE "itinerary_legs" (
  "itineraryId" INTEGER,
  "itineraryLegId" INTEGER PRIMARY KEY,
  "legDate" TEXT,
  "name" TEXT,
  "order" INTEGER,
  "startListId" INTEGER,
  "status" TEXT,
  FOREIGN KEY ("itineraryId") REFERENCES "itinerary_event" ("itineraryId")
);
CREATE TABLE "itinerary_sections" (
  "itineraryLegId" INTEGER,
  "itinerarySectionId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "order" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_stages" (
  "code" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "name" TEXT,
  "number" INTEGER,
  "stageId" INTEGER PRIMARY KEY,
  "stageType" TEXT,
  "status" TEXT,
  "timingPrecision" TEXT,
  "itineraryLegId" INTEGER,
  "itinerarySections.itinerarySectionId" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_controls" (
  "code" TEXT,
  "controlId" INTEGER PRIMARY KEY,
  "controlPenalties" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "firstCarDueDateTime" TEXT,
  "firstCarDueDateTimeLocal" TEXT,
  "location" TEXT,
  "stageId" INTEGER,
  "status" TEXT,
  "targetDuration" TEXT,
  "targetDurationMs" INTEGER,
  "timingPrecision" TEXT,
  "type" TEXT,
  "itineraryLegId" INTEGER,
  "itinerarySections.itinerarySectionId" INTEGER,
  "roundingPolicy" TEXT,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "startlists" (
  "codriver.abbvName" TEXT,
  "codriver.code" TEXT,
  "codriver.country.countryId" INTEGER,
  "codriver.country.iso2" TEXT,
  "codriver.country.iso3" TEXT,
  "codriver.country.name" TEXT,
  "codriver.countryId" INTEGER,
  "codriver.firstName" TEXT,
  "codriver.fullName" TEXT,
  "codriver.lastName" TEXT,
  "codriver.personId" INTEGER,
  "codriverId" INTEGER,
  "driver.abbvName" TEXT,
  "driver.code" TEXT,
  "driver.country.countryId" INTEGER,
  "driver.country.iso2" TEXT,
  "driver.country.iso3" TEXT,
  "driver.country.name" TEXT,
  "driver.countryId" INTEGER,
  "driver.firstName" TEXT,
  "driver.fullName" TEXT,
  "driver.lastName" TEXT,
  "driver.personId" INTEGER,
  "driverId" INTEGER,
  "eligibility" TEXT,
  "entrant.entrantId" INTEGER,
  "entrant.logoFilename" TEXT,
  "entrant.name" TEXT,
  "entrantId" INTEGER,
  "entryId" INTEGER PRIMARY KEY,
  "eventId" INTEGER,
  "group.name" TEXT,
  "groupId" INTEGER,
  "group.groupId" INTEGER,
  "identifier" TEXT,
  "manufacturer.logoFilename" TEXT,
  "manufacturer.manufacturerId" INTEGER,
  "manufacturer.name" TEXT,
  "manufacturerId" INTEGER,
  "priority" TEXT,
  "status" TEXT,
  "tag" TEXT,
  "tag.name" TEXT,
  "tag.tagId" INTEGER,
  "tagId" INTEGER,
  "tyreManufacturer" TEXT,
  "vehicleModel" TEXT,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "roster" (
  "fiasn" INTEGER,
  "code" TEXT,
  "sas-entryid" INTEGER PRIMARY KEY,
  "roster_num" INTEGER,
  FOREIGN KEY ("sas-entryid") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "startlist_classes" (
  "eventClassId" INTEGER,
  "eventId" INTEGER,
  "name" TEXT,
  "entryId" INTEGER,
  PRIMARY KEY ("eventClassId","entryId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "penalties" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "penaltyDuration" TEXT,
  "penaltyDurationMs" INTEGER,
  "penaltyId" INTEGER PRIMARY KEY,
  "reason" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "retirements" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "reason" TEXT,
  "retirementDateTime" TEXT,
  "retirementDateTimeLocal" TEXT,
  "retirementId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stagewinners" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "stageId" INTEGER,
  "stageName" TEXT,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId")
);
CREATE TABLE "stage_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "split_times" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "splitDateTime" TEXT,
  "splitDateTimeLocal" TEXT,
  "splitPointId" INTEGER,
  "splitPointTimeId" INTEGER PRIMARY KEY,
  "stageTimeDuration" TEXT,
  "stageTimeDurationMs" REAL,
  "startDateTime" TEXT,
  "startDateTimeLocal" TEXT,
  "stageId" INTEGER,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_stage" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "position" INTEGER,
  "source" TEXT,
  "stageId" INTEGER,
  "stageTimeId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "championship_lookup" (
  "championshipId" INTEGER PRIMARY KEY,
  "fieldFiveDescription" TEXT,
  "fieldFourDescription" TEXT,
  "fieldOneDescription" TEXT,
  "fieldThreeDescription" TEXT,
  "fieldTwoDescription" TEXT,
  "name" TEXT,
  "seasonId" INTEGER,
  "type" TEXT,
  "_codeClass" TEXT,
  "_codeTyp" TEXT
);
CREATE TABLE "championship_results" (
  "championshipEntryId" INTEGER,
  "championshipId" INTEGER,
  "dropped" INTEGER,
  "eventId" INTEGER,
  "pointsBreakdown" TEXT,
  "position" INTEGER,
  "publishedStatus" TEXT,
  "status" TEXT,
  "totalPoints" INTEGER,
  PRIMARY KEY ("championshipEntryId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_entries_codrivers" (
  "championshipEntryId" INTEGER PRIMARY KEY,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "CountryISO2" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_entries_manufacturers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" INTEGER,
  "Name" TEXT,
  "LogoFileName" TEXT,
  "Manufacturer" TEXT,
  "manufacturerId" INTEGER,
  "personId" TEXT,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_rounds" (
  "championshipId" INTEGER,
  "eventId" INTEGER,
  "order" INTEGER,
  PRIMARY KEY ("championshipId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_events" (
  "categories" TEXT,
  "clerkOfTheCourse" TEXT,
  "country.countryId" INTEGER,
  "country.iso2" TEXT,
  "country.iso3" TEXT,
  "country.name" TEXT,
  "countryId" INTEGER,
  "eventId" INTEGER PRIMARY KEY,
  "finishDate" TEXT,
  "location" TEXT,
  "mode" TEXT,
  "name" TEXT,
  "organiserUrl" TEXT,
  "slug" TEXT,
  "startDate" TEXT,
  "stewards" TEXT,
  "surfaces" TEXT,
  "templateFilename" TEXT,
  "timeZoneId" TEXT,
  "timeZoneName" TEXT,
  "timeZoneOffset" INTEGER,
  "trackingEventId" INTEGER ,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_entries_drivers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "CountryISO2" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "event_metadata" (
  "_id" TEXT,
  "availability" TEXT,
  "date-finish" TEXT,
  "date-start" TEXT,
  "gallery" TEXT,
  "hasdata" TEXT,
  "hasvideos" TEXT,
  "id" TEXT,
  "info-based" TEXT,
  "info-categories" TEXT,
  "info-date" TEXT,
  "info-flag" TEXT,
  "info-surface" TEXT,
  "info-website" TEXT,
  "kmlfile" TEXT,
  "logo" TEXT,
  "name" TEXT,
  "org-website" TEXT,
  "poi-Klo im Wald" TEXT,
  "poilistid" TEXT,
  "position" TEXT,
  "rosterid" TEXT,
  "sas-eventid" TEXT,
  "sas-itineraryid" TEXT,
  "sas-rallyid" TEXT,
  "sas-trackingid" TEXT,
  "sitid" TEXT,
  "testid" TEXT,
  "thumbnail" TEXT,
  "time-zone" TEXT,
  "tzoffset" TEXT,
  "year" INTEGER
);


'''


#conn = sqlite3.connect('wrc18_test1keys.db')
#c = conn.cursor()
#c.executescript(setup_q)

In [216]:
setup_views_q = '''
'''

In [165]:
#meta={'rallyId':None, 'stages':[], 'championshipId':None }

In [166]:
def getRally_URLs(results_main_url=None):
    if results_main_url is None:
        results_main_url='http://www.wrc.com/en/wrc/results/wales/stage-times/page/416-238---.html#'

    html=requests.get(results_main_url)
    soup=BeautifulSoup(html.content, "html5lib")
    #BeautifulSoup has a routine - find_all() - that will find all the HTML tags of a particular sort
    #Links are represented in HTML pages in the form <a href="http//example.com/page.html">link text</a>
    #Grab all the <a> (anchor) tags...
    souplist=soup.findAll("li",{'class':'flag'})

    items={}
    for s in souplist:
        href=s.find('a')['href']
        if href:
            title=s.find('img')['title']
            title = 'Monaco' if title == 'Monte Carlo' else title
            items[title]=href
    return items

def listRallies(display=True, **kwargs):
    rallyURLs = getRally_URLs(**kwargs)
    if display:
        print( ', '.join(rallyURLs.keys()) )
    else:
        return getRallyIDs

In [167]:
def _getEventMetadata():
    url='https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Event%22&maxdepth=1'
    eventmeta = requests.get(url).json()
    return eventmeta

def getEventMetadata():
    eventMetadata = json_normalize(_getEventMetadata(),
                                   record_path='_meta',
                                   meta='_id'  ).drop_duplicates().pivot('_id', 'n','v').reset_index()

    eventMetadata['date-finish']=pd.to_datetime(eventMetadata['date-finish'])
    eventMetadata['date-start']=pd.to_datetime(eventMetadata['date-start'])
    eventMetadata['year'] = eventMetadata['date-start'].dt.year
    return eventMetadata

In [168]:
#listRallies()

In [169]:
def _getRallyID(rallyURL):
    html=requests.get(rallyURL)
    m = re.search("var rallyId = '(.+?)'", html.text)
    if m:
        return m.group(1)
    return None
            
def getRallyIDs(rally=None,results_main_url=None):
    rallyids={}

    items = getRally_URLs(results_main_url)
    
    #if we know the rally, just get that one.. 
    if rally in items:
        items = {rally:items[rally]}

    for item in items:
        rallyids[item] = _getRallyID(items[item])

    return rallyids 

In [205]:
def _getRallyIDs2(year=2018):
    em=getEventMetadata()
    em = em[em['year']==year][['name','sas-rallyid', 'kmlfile', 'date-start']].reset_index(drop=True).dropna()
    em['stub']=em['kmlfile'].apply(lambda x: x.split('_')[0])
    return em

def getRallyIDs2(year=2018):
    em = _getRallyIDs2(year=2018)
    return em[['stub','sas-rallyid']].set_index('stub').to_dict()['sas-rallyid']

def listRallies2(year=2018):
    return getRallyIDs2(year)

In [206]:
#listRallies2()

{'sardegna': '36',
 'france': '33',
 'finland': '37',
 'portugal': '35',
 'mexico': '32',
 'argentina': '34',
 'montecarlo': '30',
 'sweden': '31'}

In [171]:
#getRallyIDs2()

{'sardegna': '36',
 'france': '33',
 'finland': '37',
 'portugal': '35',
 'mexico': '32',
 'argentina': '34',
 'montecarlo': '30',
 'sweden': '31'}

In [172]:
#getRallyIDs()

{'Monaco': '30',
 'Sweden': '31',
 'Mexico': '32',
 'France': '33',
 'Argentina': '34',
 'Portugal': '35',
 'Italy': '36',
 'Finland': '37',
 'Germany': '38',
 'Turkey': '39',
 'Wales': '40',
 'Spain': '41',
 'Australia': '42'}

In [173]:
#rallyIDs = getRallyIDs()

In [175]:
def nvToDict(nvdict, key='n',val='v', retdict=None):
    if retdict is None:
        retdict={nvdict[key]:nvdict[val]}
    else:
        retdict[nvdict[key]]=nvdict[val]
    return retdict
#assert nvToDict({'n': "id",'v': "adac-rallye-deutschland"}) == {'id': 'adac-rallye-deutschland'}

In [177]:
 #getEventMetadata()['rosterid'].iloc[0]

In [209]:
def set_rallyId(rally, year, rallyIDs=None):
    meta={'rallyId':None, 'stages':[], 'championshipId':None }
    if rallyIDs is None:
        rallyIDs = getRallyIDs()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally
    return meta

def set_rallyId2(rally, year, rallyIDs=None):
    meta={'rallyId':None, 'stages':[], 'championshipId':None }
    if rallyIDs is None:
        rallyIDs = getRallyIDs2()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally
    return meta

In [179]:
roster_id='bab64d15-4691-4561-a6bf-7284f3bd85f9'
import requests
#roster_json = requests.get( '{}&maxdepth=2'.format(wrcapi.format(roster_id),) ).json()
#roster_json   

#TO CHECK - is the sas-entryid the entryid we use elsewhere?

#This comes from event metadata
def _getRoster(roster_id):
    roster_json = requests.get(wrcapi.format(roster_id) ).json()
    roster=json_normalize(roster_json)
    
    aa=json_normalize(roster_json, record_path='_dchildren')
    zz=json_normalize(roster_json['_dchildren'],record_path=['_meta'], meta='_id').pivot('_id', 'n','v').reset_index()
    zz=pd.merge(zz,aa[['_id','name','type']], on='_id')[['fiasn','filename','sas-entryid','name']]
    zz.columns = ['fiasn','code','sas-entryid','roster_num']
    return zz

def getRoster(meta):
    em = getEventMetadata()
    roster_id= em[em['sas-rallyid']==meta['rallyId']]['rosterid'].iloc[0]
    return _getRoster(roster_id)

In [180]:
#getRoster(set_rallyId("Finland",2018))

In [242]:
def getItinerary(meta):
    ''' Get event itinerary. Also updates the stages metadata. '''
    itinerary_json=requests.get( url_base.format(stub=stubs['itinerary'].format(**meta) ) ).json()
    itinerary_event = json_normalize(itinerary_json).drop('itineraryLegs', axis=1)
    
    #meta='eventId' for eventId
    itinerary_legs = json_normalize(itinerary_json, 
                                    record_path='itineraryLegs').drop('itinerarySections', axis=1)
    #meta='eventId' for eventId
    itinerary_sections = json_normalize(itinerary_json,
                                        ['itineraryLegs', 'itinerarySections']).drop(['stages','controls'],axis=1)

    itinerary_stages=json_normalize(itinerary_json['itineraryLegs'],
                                    ['itinerarySections','stages'],
                                   meta=['itineraryLegId',['itinerarySections','itinerarySectionId']])
    meta['stages']=itinerary_stages['stageId'].tolist()
    #Should do this a pandas idiomatic way
    #meta['_stages']=zip(itinerary_stages['stageId'].tolist(),
     #                   itinerary_stages['code'].tolist(),
     #                   itinerary_stages['status'].tolist())
    meta['_stages'] = itinerary_stages[['stageId','code','status']].set_index('code').to_dict(orient='index')
    itinerary_controls=json_normalize(itinerary_json['itineraryLegs'], 
                                  ['itinerarySections','controls'] ,
                                     meta=['itineraryLegId',['itinerarySections','itinerarySectionId']])
    itinerary_controls['stageId'] = itinerary_controls['stageId'].fillna(-1).astype(int)
    
    return itinerary_event, itinerary_legs, itinerary_sections, itinerary_stages, itinerary_controls

In [182]:
#a,b,c,d,e = getItinerary(meta)

In [183]:
def _get_single_json_table(meta, stub):
    _json = requests.get( url_base.format(stub=stubs[stub].format(**meta) ) ).json()
    return json_normalize(_json)

In [184]:
#meta =  set_rallyId(name, year)

#startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
#ff=[]
#for f in startlists_json:
#    if f['manufacturer']['logoFilename'] is None:
#        f['manufacturer']['logoFilename']=''
#    if f['entrant']['logoFilename'] is None:
#        f['entrant']['logoFilename']='' 
#    ff.append(f)
#ff

In [185]:
#startlists = json_normalize(ff).drop('eventClasses', axis=1)

In [186]:
def get_startlists(meta):
    startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
    ff=[]
    for f in startlists_json:
        if f['manufacturer']['logoFilename'] is None:
            f['manufacturer']['logoFilename']=''
        if f['entrant']['logoFilename'] is None:
            f['entrant']['logoFilename']='' 
        ff.append(f)
    startlists = json_normalize(ff).drop('eventClasses', axis=1)
    startlist_classes = json_normalize(ff,['eventClasses'], 'entryId' )
    #startlists = json_normalize(startlists_json).drop('eventClasses', axis=1)
    #startlist_classes = json_normalize(startlists_json,['eventClasses'], 'entryId' )
    
    return startlists, startlist_classes 

In [187]:
def get_penalties(meta):
    ''' Get the list of penalties for a specified event. '''
    penalties = _get_single_json_table(meta, 'penalties')
    return penalties

In [188]:
def get_retirements(meta):
    ''' Get the list of retirements for a specified event. '''
    retirements = _get_single_json_table(meta, 'retirements')
    return retirements

In [189]:
def get_stagewinners(meta):
    ''' Get the stage winners table for a specified event. '''
    stagewinners = _get_single_json_table(meta, 'stagewinners')
    return stagewinners

In [261]:
def _single_stage(meta2, stub, stageId):
    ''' For a single stageId, get the requested resource. '''
    meta2['stageId']=stageId
    _json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
    _df = json_normalize(_json)
    _df['stageId'] = stageId
    return _df

def _stage_iterator(meta, stub, stage=None):
    ''' Iterate through a list of stageId values and get requested resource. '''
    meta2={'rallyId':meta['rallyId']}
    df = pd.DataFrame()
    #If stage is None get data for all stages
    if stage is not None:
        stages=[]
        #If we have a single stage (specified in form SS4) get it
        if isinstance(stage,str) and stage in meta['_stages']:
            stages.append(meta['_stages'][stage]['stageId'])
        #If we have a list of stages (in form ['SS4','SS5']) get them all
        elif isinstance(stage, list):
            for _stage in stage:
                if isinstance(_stage,str) and _stage in meta['_stages']:
                    stages.append(meta['_stages'][_stage]['stageId'])
                elif _stage in meta['stages']:
                    stages.append(_stage)
    else:
        stages = meta['stages']
        
    #Get data for required stages
    for stageId in stages:
        #meta2['stageId']=stageId
        #_json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
        #_df = json_normalize(_json)
        #_df['stageId'] = stageId
        _df = _single_stage(meta2, stub, stageId)
        df = pd.concat([df, _df])
    return df.reset_index(drop=True)

In [255]:
def get_overall(meta, stage=None):
    ''' Get the overall results table for all stages on an event or a specified stage. '''
    stage_overall = _stage_iterator(meta, 'overall', stage)
    return stage_overall

In [253]:
def get_splitTimes(meta, stage=None):
    split_times = _stage_iterator(meta, 'split_times', stage)
    return split_times

In [193]:
def get_stage_times_stage(meta, stage=None):
    stage_times_stage = _stage_iterator(meta, 'stage_times_stage', stage)
    return stage_times_stage

In [194]:
def get_stage_times_overall(meta,stage=None):
    stage_times_overall = _stage_iterator(meta, 'stage_times_overall', stage)
    return stage_times_overall

In [195]:
def _get_championship_codes(url=None):
    if url is None:
        url = 'http://www.wrc.com/en/wrc/results/championship-standings/page/4176----.html'
    html2=requests.get(url).text
    m = re.search("var championshipClasses = (.*?);", html2, re.DOTALL)
    mm=m.group(1).replace('\n','').replace("'",'"')
    d=json.loads(mm)
    #https://stackoverflow.com/a/35758583/454773
    championshipClasses={k.replace(' ', ''): v for k, v in d.items()}
    return championshipClasses

In [196]:
#_get_championship_codes()

In [197]:
def championship_tables(champ_class=None, champ_typ=None):
    #if championship is None then get all
    championship_lookup = pd.DataFrame()
    championship_entries_all = {}
    championship_rounds = pd.DataFrame()
    championship_events = pd.DataFrame()
    championship_results = pd.DataFrame()
    
    championship_codes = _get_championship_codes()
    _class_codes = championship_codes.keys() if champ_class is None else [champ_class]
    for champClass in _class_codes:
        _champ_typ = championship_codes[champClass].keys() if champ_typ is None else [champ_typ]
        for champType in _champ_typ:
            if champType not in championship_entries_all:
                championship_entries_all[champType] = pd.DataFrame()
            
            champ_num = championship_codes[champClass][champType]
            meta2={'championshipId': champ_num}
            
            championship_json=requests.get( url_base.format(stub=stubs['championship'].format(**meta2) ) ).json()

            _championship_lookup = json_normalize(championship_json).drop(['championshipEntries','championshipRounds'], axis=1)
            _championship_lookup['_codeClass'] = champClass
            _championship_lookup['_codeTyp'] = champType
            championship_lookup = pd.concat([championship_lookup,_championship_lookup],sort=True)
            
            championships={}
            championship_dict = _championship_lookup.to_dict()
            championships[champ_num] = {c:championship_dict[c][0] for c in championship_dict}
            renamer={c.replace('Description',''):championships[champ_num][c] for c in championships[champ_num] if c.startswith('field')}            
            _championship_entries = json_normalize(championship_json,['championshipEntries'] )
            _championship_entries = _championship_entries.rename(columns=renamer)
            _championship_entries = _championship_entries[[c for c in _championship_entries.columns if c!='']]
            #pd.concat sort=False to retain current behaviour
            championship_entries_all[champType] = pd.concat([championship_entries_all[champType],_championship_entries],sort=False)
            
            _championship_rounds = json_normalize(championship_json,['championshipRounds'] ).drop('event', axis=1)
            championship_rounds = pd.concat([championship_rounds,_championship_rounds],sort=False).drop_duplicates()
            
            _events_json = json_normalize(championship_json,['championshipRounds' ])['event']
            _championship_events = json_normalize(_events_json)
            championship_events = pd.concat([championship_events,_championship_events],sort=False).drop_duplicates()
        
            _championship_results = _get_single_json_table(meta2, 'championship_results')
            championship_results = pd.concat([championship_results, _championship_results],sort=False)
    
    for k in championship_entries_all:
        championship_entries_all[k].reset_index(drop=True)
        if k in ['Driver', 'Co-Driver']:
            championship_entries_all[k] = championship_entries_all[k].rename(columns={'TyreManufacturer':'ManufacturerTyre'})
    
    return championship_lookup.reset_index(drop=True), \
            championship_results.reset_index(drop=True), \
            championship_entries_all, \
            championship_rounds.reset_index(drop=True), \
            championship_events.reset_index(drop=True)


In [198]:
#a,b,c,d,e = championship_tables()

## Usage

In [199]:
#listRallies()

In [200]:
def cleardbtable(conn, table):
    ''' Clear the table whilst retaining the table definition '''
    c = conn.cursor()
    c.execute('DELETE FROM "{}"'.format(table))
    
def dbfy(conn, df, table, if_exists='append',index=False, clear=False, **kwargs):
    if clear: cleardbtable(conn, table)
    df.to_sql(table,conn,if_exists=if_exists,index=index)

In [225]:
def save_rally(meta, conn):
    
    roster = getRoster(meta)
    dbfy(conn, roster, 'roster')
    
    itinerary_event, itinerary_legs, itinerary_sections, \
    itinerary_stages, itinerary_controls = getItinerary(meta)
    
    dbfy(conn, itinerary_event, 'itinerary_event')
    dbfy(conn, itinerary_legs, 'itinerary_legs')
    dbfy(conn, itinerary_sections, 'itinerary_sections')
    dbfy(conn, itinerary_stages, 'itinerary_stages')
    dbfy(conn, itinerary_controls, 'itinerary_controls')

    startlists, startlist_classes = get_startlists(meta)
    dbfy(conn, startlists, 'startlists')
    dbfy(conn, startlist_classes, 'startlist_classes')

    penalties = get_penalties(meta)
    dbfy(conn, penalties, 'penalties')

    retirements = get_retirements(meta)
    dbfy(conn, retirements, 'retirements')

    stagewinners = get_stagewinners(meta)
    dbfy(conn, stagewinners, 'stagewinners')


    stage_overall = get_overall(meta)
    dbfy(conn, stage_overall, 'stage_overall')

    split_times = get_splitTimes(meta)
    dbfy(conn, split_times, 'split_times')
    
    stage_times_stage = get_stage_times_stage(meta)
    dbfy(conn, stage_times_stage, 'stage_times_stage')
    
    stage_times_overall = get_stage_times_overall(meta)
    dbfy(conn, stage_times_overall, 'stage_times_overall')

    
def save_championship(meta, conn):
    championship_lookup, championship_results, _championship_entries_all, \
        championship_rounds, championship_events = championship_tables()
        
    championship_entries_drivers = _championship_entries_all['Driver']
    championship_entries_codrivers = _championship_entries_all['Co-Driver']
    championship_entries_manufacturers = _championship_entries_all['Manufacturers']
    
    dbfy(conn, championship_lookup, 'championship_lookup', clear=True)
    dbfy(conn, championship_results, 'championship_results', clear=True)
    dbfy(conn, championship_entries_drivers, 'championship_entries_drivers', clear=True)
    dbfy(conn, championship_entries_codrivers, 'championship_entries_codrivers', clear=True)
    dbfy(conn, championship_entries_manufacturers, 'championship_entries_manufacturers', clear=True)
    dbfy(conn, championship_rounds, 'championship_rounds', clear=True)
    dbfy(conn, championship_events, 'championship_events', clear=True)

def get_all(rally, dbname='wrc18_test1.db', year=2018):
    
    conn = sqlite3.connect(dbname)
    
    meta =  set_rallyId(rally, year)
    
    save_rally(meta, conn)
    save_championship(meta, conn)


In [243]:
_meta=set_rallyId(name, year)
_,_,_,_s,_ = getItinerary(_meta)
_meta

{'rallyId': '37',
 'stages': [660,
  658,
  655,
  676,
  654,
  667,
  668,
  673,
  664,
  662,
  674,
  672,
  671,
  670,
  669,
  666,
  665,
  663,
  675,
  656,
  661,
  657,
  659],
 'championshipId': None,
 'rally_name': 'Finland',
 '_stages': {'SS1': {'stageId': 660, 'status': 'Completed'},
  'SS2': {'stageId': 658, 'status': 'Completed'},
  'SS3': {'stageId': 655, 'status': 'Completed'},
  'SS4': {'stageId': 676, 'status': 'Completed'},
  'SS5': {'stageId': 654, 'status': 'Completed'},
  'SS6': {'stageId': 667, 'status': 'Interrupted'},
  'SS7': {'stageId': 668, 'status': 'Completed'},
  'SS8': {'stageId': 673, 'status': 'Completed'},
  'SS9': {'stageId': 664, 'status': 'Completed'},
  'SS10': {'stageId': 662, 'status': 'Completed'},
  'SS11': {'stageId': 674, 'status': 'Completed'},
  'SS12': {'stageId': 672, 'status': 'Completed'},
  'SS13': {'stageId': 671, 'status': 'Completed'},
  'SS14': {'stageId': 670, 'status': 'Interrupted'},
  'SS15': {'stageId': 669, 'status': 'C

In [207]:
#listRallies()
listRallies2()

{'sardegna': '36',
 'france': '33',
 'finland': '37',
 'portugal': '35',
 'mexico': '32',
 'argentina': '34',
 'montecarlo': '30',
 'sweden': '31'}

In [68]:
#rr=get_retirements(meta)
#rr.head()

In [69]:
#dbname='wrc18.db'
year = 2018
name = 'Finland'
dbname='finland18.db'

In [70]:
# TO DO - ability to top up just the stage we need

In [231]:
#set_rallyId("Finland",2018)
set_rallyId2("finland",2018)

{'rallyId': '37',
 'stages': [],
 'championshipId': None,
 'rally_name': 'finland'}

In [223]:
#new db
!rm $dbname
conn = sqlite3.connect(dbname)
c = conn.cursor()
c.executescript(setup_q)
c.executescript(setup_views_q)
q="SELECT name FROM sqlite_master WHERE type = 'table';"

dbfy(conn, getEventMetadata(), 'event_metadata')

pd.read_sql(q,conn)

/Users/ajh59/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,roster
7,startlist_classes
8,penalties
9,retirements


In [136]:
#itinerary_event

In [137]:
#itinerary_event

In [226]:
#set_rallyId( name,year )
get_all(name, dbname=dbname, year=year )

In [139]:
meta

{'rallyId': '36', 'stages': [], 'championshipId': None, 'rally_name': 'Italy'}

In [228]:
meta

NameError: name 'meta' is not defined

In [219]:
conn = sqlite3.connect(dbname)

q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,roster
7,startlist_classes
8,penalties
9,retirements


In [221]:
q="SELECT * FROM event_metadata LIMIT 1;"
pd.read_sql(q,conn).columns

Index(['_id', 'availability', 'date-finish', 'date-start', 'gallery',
       'hasdata' TEXT,\n  'hasvideos', 'id', 'info-based', 'info-categories',
       'info-date', 'info-flag', 'info-surface', 'info-website', 'kmlfile',
       'logo', 'name', 'org-website', 'poi-Klo im Wald', 'poilistid',
       'position', 'rosterid', 'sas-eventid', 'sas-itineraryid', 'sas-rallyid',
       'sas-trackingid', 'sitid', 'testid', 'thumbnail', 'time-zone',
       'tzoffset', 'year'],
      dtype='object')

In [ ]:
#!rm wrc18_test1.db

In [41]:
q="SELECT * FROM stage_times_overall LIMIT 10;"
pd.read_sql(q,conn)

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId


In [227]:
!ls -al *.db

-rw-r--r--  1 ajh59  1182653967    36864 12 Feb 15:32 amagicdemo.db
-rw-r--r--  1 ajh59  1182653967   638976 19 May 16:01 argentina18.db
-rw-r--r--  1 ajh59  1182653967        0 10 Feb 18:32 database.db
-rw-r--r--  1 ajh59  1182653967  1421312 31 Jul 14:08 finland18.db
-rw-r--r--  1 ajh59  1182653967   974848 19 May 12:46 france18.db
-rw-r--r--  1 ajh59  1182653967        0  8 Jun 08:30 italy.db
-rw-r--r--  1 ajh59  1182653967  1024000 11 Jun 09:46 italy18.db
-rw-r--r--  1 ajh59  1182653967   622592 19 May 12:45 mexico18.db
-rw-r--r--  1 ajh59  1182653967   966656 19 May 12:44 monaco18.db
-rw-r--r--  1 ajh59  1182653967  1290240 20 May 14:30 portugal18.db
-rw-r--r--  1 ajh59  1182653967        0  3 Feb 20:55 rally.db
-rw-r--r--  1 ajh59  1182653967  1236992 19 May 12:45 sweden18.db
-rw-r--r--  1 ajh59  1182653967  2117632  5 Mar 21:02 wrc18.db
-rw-r--r--  1 ajh59  1182653967   954368  5 Mar 15:33 wrc18_monaco.db
-rw-r--r--  1 ajh59  1182653967   954368  5 Mar 15:31 wrc18_sweden.db
-rw-

In [133]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(dbname)
c = conn.cursor()
#c.executescript(setup_q)
#c.executescript(setup_views_q)
q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,startlist_classes
7,penalties
8,retirements
9,stagewinners


In [134]:
q="SELECT * FROM startlists LIMIT 1;"
pd.read_sql(q,conn).to_dict()

{'codriver.abbvName': {0: 'J. INGRASSIA'},
 'codriver.code': {0: 'ING'},
 'codriver.country.countryId': {0: 76},
 'codriver.country.iso2': {0: 'FR'},
 'codriver.country.iso3': {0: 'FRA'},
 'codriver.country.name': {0: 'France'},
 'codriver.countryId': {0: 76},
 'codriver.firstName': {0: 'Julien'},
 'codriver.fullName': {0: 'Julien INGRASSIA'},
 'codriver.lastName': {0: 'INGRASSIA'},
 'codriver.personId': {0: 521},
 'codriverId': {0: 521},
 'driver.abbvName': {0: 'S. OGIER'},
 'driver.code': {0: 'OGI'},
 'driver.country.countryId': {0: 76},
 'driver.country.iso2': {0: 'FR'},
 'driver.country.iso3': {0: 'FRA'},
 'driver.country.name': {0: 'France'},
 'driver.countryId': {0: 76},
 'driver.firstName': {0: 'Sébastien\xa0'},
 'driver.fullName': {0: 'Sébastien\xa0 OGIER'},
 'driver.lastName': {0: 'OGIER'},
 'driver.personId': {0: 2453},
 'driverId': {0: 2453},
 'eligibility': {0: 'M'},
 'entrant.entrantId': {0: 94},
 'entrant.logoFilename': {0: ''},
 'entrant.name': {0: 'M-SPORT FORD WORLD RA